In [52]:
import os
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from datetime import datetime
from collections import Counter
from nltk import tokenize
import json

In [53]:
# Folder where the xml files are located
FILE_DIR = "Input_files"

In [54]:
# Reads in the filenames, and prints out how many start with "oai"

filename_list = []
for filename in os.listdir(FILE_DIR):
    if filename.startswith('oai'):
        filename_list.append(filename)
print(len(filename_list))

1646


In [55]:
# DKRZ use a number of tags for references.  These variables store the relevant tags against the agreed variable 
# in the IPCC DDC schema v2

references_rel_types = ['IsDocumentedBy', 'Cites', 'IsDescribedBy', 'References']
referencedby_rel_types = ['Documents', 'IsCitedBy', 'Describes', 'IsReferencedBy']

## XML IMPORT AND MAPPING

In [57]:
# This script reads in the files and creates a list of python dictionaries, each one containing the mapped 
# metadata from each XML file

import_list = []
for xml_file in filename_list:
    import_dict = {}
    etree = ET.parse(os.path.join(FILE_DIR,xml_file))
    root = etree.getroot()
    
    #### SUMMARY
    
    # Title
    for titles in root.findall('{http://datacite.org/schema/kernel-4}titles'):
        import_dict['Title'] = titles.find('{http://datacite.org/schema/kernel-4}title').text
    
    #Keywords
    keyword_list = []
    for keywords in root.findall('{http://datacite.org/schema/kernel-4}subjects'):
        for keyword in keywords.findall('{http://datacite.org/schema/kernel-4}subject'):
                                       keyword_list.append(keyword.text)
    import_dict['Keywords'] = keyword_list
    
    
    
    # DOI name, or alternative indentifiers
    
    '''
    For DOIs, DKRZ provided the following detail on how to create resolvable URLS from the DOIs:
    
    We have persistent urls pointing to the landing page, which provide information about and access to the data. Unfortunately, we haven't these urls in the provided metadata and there are two construction rules based on the 'doi name'.
    
    special case for doi names with 'CMIP5.' 
    or which are like '10.1594/WDCC/CMIP5.': http://cera-www.dkrz.de/WDCC/CMIP5/Compact.jsp?acronym= 
    e.g. 10.1594/WDCC/CMIP5.MXELr4: http://cera-www.dkrz.de/WDCC/CMIP5/Compact.jsp?acronym=MXELr4
    
    all other cases, currently with 'doi names' like '10.1594/WDCC/ or '10.26050/WDCC/': 
    http://cera-www.dkrz.de/WDCC/ui/Compact.jsp?acronym= 
    e.g. '10.1594/WDCC/ETHr8': http://cera-www.dkrz.de/WDCC/ui/Compact.jsp?acronym=10.1594/WDCC/ETHr8
    
    Alternatively, we could use DOI resolver+DOI name: https://doi.org/
    
    '''
    
    
    for identifier in root.findall('{http://datacite.org/schema/kernel-4}identifier'):
        id_type = identifier.get('identifierType')
        if id_type == "DOI":
            import_dict['DOI Name'] = identifier.text
            import_dict['Alternate Identifier'] = ''
            if identifier.text.startswith('10.1594/WDCC/CMIP5.'):
                acronym = identifier.text.split('10.1594/WDCC/CMIP5.', 1)[1]
                import_dict['Access URL'] = 'http://cera-www.dkrz.de/WDCC/CMIP5/Compact.jsp?acronym={}'.format(acronym)
            else:
                acronym = identifier.text.split('/WDCC/', 1)[1]
                import_dict['Access URL'] = 'http://cera-www.dkrz.de/WDCC/ui/Compact.jsp?acronym={}'.format(acronym)
        else:
            import_dict['DOI Name'] = ''
            import_dict['Alternate Identifier'] = identifier.text
            import_dict['Access URL'] = identifier.text
            
    # Publication date
    for dates in root.findall('{http://datacite.org/schema/kernel-4}dates'):
        for date in dates.findall('{http://datacite.org/schema/kernel-4}date'):
            date_type = date.get('dateType')
            if date_type == "Created":
                import_dict['Publication Date'] = date.text
    #Publication year
    for year in root.findall('{http://datacite.org/schema/kernel-4}publicationYear'):
        import_dict['Publication Year'] = year.text
        
    
    #### PUBLISHER 
    for publisher in root.findall('{http://datacite.org/schema/kernel-4}publisher'):
        import_dict['Pub_Name'] = publisher.text
        
    #### DOCUMENTATION
    
    for descs in root.findall('{http://datacite.org/schema/kernel-4}descriptions'):
        for desc in descs.findall('{http://datacite.org/schema/kernel-4}description'):
            if desc.get('descriptionType') == "Abstract":
                desc_text = desc.text
                for summary in desc.iter():
                    desc_text += summary.tail
                    text = summary.tail
                    if text.split(':')[0] == "Summary":
                        # Using the first sentense of the summary section for the Abstract
                        abstract_text = tokenize.sent_tokenize(text.split(':', 1)[1])[0]
                        if len(abstract_text) > 180:
                            abstract_text = abstract_text.split(')')[0]
                        import_dict['Abstract'] = abstract_text
                        
                import_dict['Description'] = desc_text 
    
    #### Coverage
    for geo_locs in root.findall('{http://datacite.org/schema/kernel-4}geoLocations'):
        for geo_loc in geo_locs.findall('{http://datacite.org/schema/kernel-4}geoLocation'):
            for geo_loc_place in geo_loc.findall('{http://datacite.org/schema/kernel-4}geoLocationPlace'):
                import_dict['Spatial Coverage'] = geo_loc_place.text
            for geo_loc_bbox in geo_loc.findall('{http://datacite.org/schema/kernel-4}geoLocationBox'):
                for west_long in geo_loc_bbox.findall('{http://datacite.org/schema/kernel-4}westBoundLongitude'):
                    import_dict['Upper Right Longitude'] = west_long.text
                for east_long in geo_loc_bbox.findall('{http://datacite.org/schema/kernel-4}eastBoundLongitude'):
                    import_dict['Lower Left Longitude'] = east_long.text
                for south_lat in geo_loc_bbox.findall('{http://datacite.org/schema/kernel-4}southBoundLatitude'):
                    import_dict['Lower Left Latitude'] = south_lat.text
                for north_lat in geo_loc_bbox.findall('{http://datacite.org/schema/kernel-4}northBoundLatitude'):
                    import_dict['Upper Right Latitude'] = north_lat.text
            
    
    #### USAGE
    
    # License
    for rights in root.findall('{http://datacite.org/schema/kernel-4}rightsList'):
        for right in rights.findall('{http://datacite.org/schema/kernel-4}rights'):
            import_dict['License'] = right.text
    
    # Resource Creator
    name_list = []
    for creators in root.findall('{http://datacite.org/schema/kernel-4}creators'):
        for creator in creators.findall('{http://datacite.org/schema/kernel-4}creator'):
            for creator_name in creator.findall('{http://datacite.org/schema/kernel-4}creatorName'):
                name_list.append(creator_name.text)
    import_dict['Resource Creator'] = name_list
    
    #### Access
    
    # Access URL - see above section DOI name, or alternative indentifiers
    
    # language
    for language in root.findall('{http://datacite.org/schema/kernel-4}language'):
        import_dict['Language'] = language.text
        
    # format
    format_list = []
    for data_formats in root.findall('{http://datacite.org/schema/kernel-4}formats'):
        for data_format in data_formats.findall('{http://datacite.org/schema/kernel-4}format'):
            format_list.append(data_format.text)
    import_dict['Format'] = format_list
    
    # references
    for reference in root.findall('{http://datacite.org/schema/kernel-4}relatedIdentifiers'):
        reference_list = []
        referenced_by_list = []
        qualified_relation_list = []
        for ri in reference.findall('{http://datacite.org/schema/kernel-4}relatedIdentifier'):
            rel_type = ri.attrib['relationType']
            if rel_type in references_rel_types:
                reference_list.append("https://doi.org/" + ri.text)
            elif rel_type in referencedby_rel_types:
                referenced_by_list.append("https://doi.org/" + ri.text)
            else:
                qualified_relation_list.append("https://doi.org/" + ri.text)
    import_dict['References'] = reference_list
    import_dict['Is Referenced By'] = referenced_by_list
    import_dict['Qualified Relation'] = qualified_relation_list
    
    #For fields with no match:
    import_dict['License'] = ''
    
    import_dict['Access Service']=''
    import_dict['Purpose'] = ''
    import_dict['Pub_Contact Point'] = 'data@dkrz.de'
    import_dict['Associated Media'] = ''
    import_dict['Is Part Of'] = ''
    import_dict['Tools'] = ''
    import_dict['Temporal Resolution']= '' 
    import_dict['Start Date'] = ''
    import_dict['Source'] = ''
    import_dict['Spatial Resolution'] = ''
    import_dict['End Date'] = ''
    import_dict['Investigations'] = '' 
    import_dict['Contact Point'] = 'data@dkrz.de'
    import_dict['Pub_Identifier'] = 'https://ror.org/03ztgj037'
    import_dict['Pub_Description'] = '' 
    import_dict['Pub_Logo'] = ''
    import_dict['Spatial Aggregation'] = '' 
    import_dict['Jurisdiction'] = ''

    import_list.append(import_dict)

In [58]:
# this script creates a dataframe in the same order as the MDW Bulk Import sheet

df = pd.DataFrame(import_list)
column_order = ['Title', 'Abstract', 'Contact Point', 'Keywords', 'DOI Name', 'Alternate Identifier',
                'Publication Date', 'Pub_Identifier', 'Pub_Name', 'Pub_Logo', 'Pub_Description', 'Pub_Contact Point', 
                'Description', 'Associated Media', 'Is Part Of', 'Spatial Coverage', 'Spatial Aggregation',
                'Spatial Resolution', 'Start Date', 'End Date', 'Temporal Resolution', 
                'Lower Left Latitude', 'Lower Left Longitude', 'Upper Right Latitude', 'Upper Right Longitude',
                'Purpose', 'Source', 'License', 'Resource Creator', 'Investigations',
                'Is Referenced By', 'References', 'Access URL', 'Access Service',
                'Jurisdiction', 'Language', 'Format', 'Qualified Relation', 'Tools']
df = df[column_order]
df

,Title,Abstract,Contact Point,Keywords,DOI Name,Alternate Identifier,Publication Date,Pub_Identifier,Pub_Name,Pub_Logo,...,Investigations,Is Referenced By,References,Access URL,Access Service,Jurisdiction,Language,Format,Qualified Relation,Tools
0,CMIP5 simulations of the Max Planck Institute ...,rcp26 is an experiment of the CMIP5 - Coupled...,data@dkrz.de,"[Climate, CMIP5, IPCC, IPCC-AR5, IPCC-AR5_CMIP...",10.1594/WDCC/CMIP5.MXMRr2,,2012-05-31,https://ror.org/03ztgj037,World Data Center for Climate (WDCC) at DKRZ,,...,,[],[https://doi.org/10.1002/jame.20038],http://cera-www.dkrz.de/WDCC/CMIP5/Compact.jsp...,,,en,[application/x-netcdf],[],
1,IPSL-CM5B-LR model output prepared for CMIP5 a...,'aquaControl' is an experiment of the CMIP5 -...,data@dkrz.de,"[Climate, CMIP5, IPCC, IPCC-AR5, IPCC-AR5_CMIP...",10.1594/WDCC/CMIP5.IPIBqc,,2013-09-16,https://ror.org/03ztgj037,World Data Center for Climate (WDCC) at DKRZ,,...,,[],[https://doi.org/10.1002/jame.20038],http://cera-www.dkrz.de/WDCC/CMIP5/Compact.jsp...,,,en,[application/x-netcdf],[],
2,HC01GG03 - GHG:GREENHOUSE GAS INTEGRATION,Here the greenhouse gas forcing is increased ...,data@dkrz.de,"[Climate, HadCM2, Hadley-Centre, IPCC, IPCC-DD...",,http://cera-www.dkrz.de/WDCC/ui/Compact.jsp?ac...,2001-11-26,https://ror.org/03ztgj037,World Data Center for Climate (WDCC) at DKRZ,,...,,[],[https://doi.org/10.1002/jame.20038],http://cera-www.dkrz.de/WDCC/ui/Compact.jsp?ac...,,,en,[application/octet-stream],[],
3,CC01GS01 - GHS: THE SULPHATE AEROSOL AND GREEN...,"(member 1 of 3 ensemble simulations, the orig...",data@dkrz.de,"[Climate, CCCma, CGCM1, IPCC, IPCC-DDC_SAR, ae...",,http://cera-www.dkrz.de/WDCC/ui/Compact.jsp?ac...,2001-11-26,https://ror.org/03ztgj037,World Data Center for Climate (WDCC) at DKRZ,,...,,[],[https://doi.org/10.1002/jame.20038],http://cera-www.dkrz.de/WDCC/ui/Compact.jsp?ac...,,,en,[application/octet-stream],[],
4,"NOAA GFDL GFDL-CM3, sstClim experiment output ...",'sstClim' is an experiment of the CMIP5 - Cou...,data@dkrz.de,"[Climate, CMIP5, GFDL-CM3, IPCC, IPCC-AR5, IPC...",10.1594/WDCC/CMIP5.NGG3sc,,2014-04-11,https://ror.org/03ztgj037,World Data Center for Climate (WDCC) at DKRZ,,...,,[],[https://doi.org/10.1175/2011JCLI3955.1],http://cera-www.dkrz.de/WDCC/CMIP5/Compact.jsp...,,,en,[application/x-netcdf],[],
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641,HadGEM2-ES model output prepared for CMIP5 esm...,"""esmrcp85"" is an experiment of the CMIP5 - Co...",data@dkrz.de,"[Climate, CMIP5, HadGEM2-ES, IPCC, IPCC-AR5, I...",10.1594/WDCC/CMIP5.MOGEe8,,2014-02-03,https://ror.org/03ztgj037,World Data Center for Climate (WDCC) at DKRZ,,...,,[],"[https://doi.org/10.5194/gmd-4-723-2011, https...",http://cera-www.dkrz.de/WDCC/CMIP5/Compact.jsp...,,,en,[application/x-netcdf],[],
1642,"cmip5 output2 MPI-M MPI-ESM-P historical, serv...",'historical' is an experiment of the CMIP5 - ...,data@dkrz.de,"[Climate, CMIP5, IPCC, IPCC-AR5, IPCC-AR5_CMIP...",,http://cera-www.dkrz.de/WDCC/CMIP5/Compact.jsp...,2014-02-14,https://ror.org/03ztgj037,World Data Center for Climate (WDCC) at DKRZ,,...,,[],"[https://doi.org/10.5194/gmd-4-723-2011, https...",http://cera-www.dkrz.de/WDCC/CMIP5/Compact.jsp...,,,en,[application/x-netcdf],[],
1643,NASA-GISS: GISS-E2-R model output prepared for...,historicalExt is an experiment of the CMIP5 -...,data@dkrz.de,"[Climate, CMIP5, GISS-E2-R, IPCC, IPCC-AR5, IP...",10.1594/WDCC/CMIP5.GIGRhx,,2013-08-29,https://ror.org/03ztgj037,World Data Center for Climate (WDCC) at DKRZ,,...,,[],"[https://doi.org/10.5194/gmd-4-723-2011, https...",http://cera-www.dkrz.de/WDCC/CMIP5/Compact.jsp...,,,en,[application/x-netcdf],[],
1644,GF01GS01 - GHS: THE HISTORICAL AND FUTURE FORC...,This simulation starts CO2 and aerosol forcin...,data@dkrz.de,"[Climate, GFDL, IPCC, IPCC-DDC, IPCC-DDC_SAR, ...",,http://cera-www.dkrz.de/WDCC/ui/Compact.jsp?ac...,2001-11-26,https://ror.org/03ztgj037,World Data Center for Climate (WDCC) at DKRZ,

# JSON Clean up

This section of the code converts python lists to JSON.

It replaces empty lists with NaN.  Note: as per the Readme, before the excel spreadsheet can be imported into the MDX, you will need to Find all "NaN" and replace with a blank cell.

In [60]:
def json_clean_column(a_pandas_series):
    
    # remove blanks
    a_pandas_series = a_pandas_series.apply(lambda x: np.nan if len(x) == 0 else x)
    
    #replace single quotes with double quotes
    a_pandas_series = a_pandas_series.apply(lambda x: json.dumps(x))
    
    return a_pandas_series

In [64]:
columns_with_lists = ['Keywords', 'Resource Creator',  'Is Referenced By', 'References', 
                      'Format', 'Qualified Relation']

In [65]:
for col in columns_with_lists:
    df[col] = json_clean_column(df[col])

In [66]:
date = datetime.date(datetime.now())
print(date)

2021-12-09


In [67]:
# This section of the code divides the records into seperate files containing no more than 
# n records per file for easy uploading

n = 100  #chunk row size
list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]

In [68]:
file_number = 1
for l_df in list_df:
    with pd.ExcelWriter('output_files/DKRZ_metadata_v1_{}_fileno_{}.xlsx'.format(date, file_number)) as writer:  
        l_df.to_excel(writer, sheet_name='DKRZ_Datasets', index=False)
    file_number +=1